In [1]:
import pyodbc
import psycopg2

## Connect to the Access database

In [2]:
def get_access_connection(access_conn_str):
    access_conn = pyodbc.connect(access_conn_str)
    return access_conn

In [3]:
access_db_file = r"C:\Users\its_soppie\Downloads\PUB150\WPI.mdb"
access_conn_str = f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={access_db_file}"
access_conn = get_access_connection(access_conn_str)
access_cursor = access_conn.cursor()

## Connect to the PostgreSQL database

In [4]:
def get_pg_connection(host, database, user, password):
    pg_conn = psycopg2.connect(host = host, database = database, user = user, password = password)
    return pg_conn

In [5]:
pg_conn = get_pg_connection('localhost', 'Capstone', 'postgres', 'Le@rn1ng')
pg_cur = pg_conn.cursor()

Test connection to Access database 

In [8]:
for i in access_cursor.tables(tableType='Table'):
    print(i.table_name)

~TMPCLP62091
Country Codes
Country Codes Old
Depth Code LUT
Drydock/Marine Railway Code LUT
Harbor Size LUT
Harbor Type LUT
Maximum Size Vessel LUT
Repairs Code LUT
Shelter Afforded LUT
Wpi Data
WPI Import
WPI Region


Execute a SELECT query to retrieve the data from the 'Wpi Data' table in Access

In [6]:
access_query = 'SELECT * FROM [Wpi data]'
access_cursor.execute(access_query)

# Fetch the column names and data types from the Access cursor description
column_names = [column[0] for column in access_cursor.description]
column_data_types = [column[1] for column in access_cursor.description]

##  Create a table in the PostgreSQL database dynamically 

 Create a data type mapping dictionary 

In [9]:
data_type_mapping = {
    'int': 'INTEGER',
    'float': 'REAL',
    'str': 'VARCHAR',
    'datetime': 'TIMESTAMP',
}

CREATE TABLE dynamically based on the column names and data types

In [ ]:
create_table_query = f'''
    CREATE TABLE wpi_data (
        {', '.join([f'"{col}" {data_type_mapping.get(dtype.__name__, "VARCHAR")}' for col, dtype in zip(column_names, column_data_types)])}
    )
'''
pg_cur.execute(create_table_query)
pg_conn.commit()

## Insert data into PostgreSQL dynamically

In [ ]:
for row in access_cursor:
    insert_query = f'''
        INSERT INTO wpi_data ({', '.join([f'"{col}"' for col in column_names])})
        VALUES ({', '.join(['%s' for _ in column_names])})
    '''
    pg_cur.execute(insert_query, row)

# Commit the changes to the PostgreSQL database
pg_conn.commit()

## Close the cursors and connections

In [ ]:
access_cursor.close()
access_conn.close()
pg_cur.close()
pg_conn.close()